<a href="https://colab.research.google.com/github/readall/indiatranslator/blob/main/indicTranslate_Transliter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
rm -rf sample_data
rm -rf fairseq
rm -rf indicTrans
rm indic-en.zip

In [ ]:
%%bash
rm -rf sample_data
pip install tqdm boto3 requests regex sentencepiece sacremoses transformers easyocr
pip install mosestokenizer indic-nlp-library mock sacrebleu tensorboardX pyarrow subword-nmt
git clone --depth 1 https://github.com/pytorch/fairseq.git
cd fairseq
pip install --editable ./
%cd ..
# pip install fairseq


In [ ]:
# clone the repo for running evaluation
!git clone --depth 1 https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone --depth 1 https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone --depth 1 https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone --depth 1 https://github.com/rsennrich/subword-nmt.git
%cd ..

In [ ]:
%%bash
# downloading the indic-indic model
wget https://storage.googleapis.com/samanantar-public/V0.3/models/indic-en.zip
unzip indic-en.zip
rm indic-en.zip
wget https://storage.googleapis.com/samanantar-public/V0.3/models/m2m.zip
unzip m2m.zip
rm m2m.zip

# %cd indicTrans

In [ ]:
!pwd
!cp /content/m2m/vocab/bpe_codes.32k.SRC_TGT /content/m2m/vocab/bpe_codes.32k.SRC


In [ ]:
%%bash
apt-get install python-numpy libicu-dev
pip install pyicu pycld2 morfessor 
pip install polyglot


In [ ]:
!polyglot download embeddings2.en transliteration2.ar
!polyglot download transliteration2.kn

In [ ]:
# we need to be in indicTrans directory for translation model to load
%pwd
%cd
%cd /content/indicTrans/
%pwd

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt  
import easyocr
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils
from indicTrans.inference.engine import Model
from polyglot.text import Text

class KannadaToHindi :
  def __init__(self):
    self.reader = easyocr.Reader(['kn','en']) # need to run only once to load model into memory
    self.indicm2m_model = Model(expdir='/content/m2m')


  def getTextFromImage(self, image_file):
    reader = easyocr.Reader(['kn','en']) # need to run only once to load model into memory
    result = reader.readtext(image_file, detail = 0)
    return result

  def cleanTransliterText(self, transliter_text):
    blank = ['',"", " ", ' ', '\t']
    s_list = []
    for x in transliter_text:
      if x not in blank:
        s_list.append(x)
    return s_list

  def getTransliterredText(self,text_from_image):
    image_text = str(text_from_image)
    text = Text(image_text)
    transliter_text = text.transliterate("hi")
    trans_str = " ".join(self.cleanTransliterText(transliter_text))
    return trans_str

  def TranslateKannadaImageToHindi(self, kannada_img_file):
    text_from_image = self.reader.readtext(kannada_img_file, detail = 0)
    transliterred_result = self.getTransliterredText(text_from_image)
    translated_result = self.indicm2m_model.batch_translate(text_from_image, 'kn', 'hi')
    return translated_result, transliterred_result


In [ ]:
kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_01.jpg'
transcode = KannadaToHindi()
translated, translittered = transcode.TranslateKannadaImageToHindi(kannada_img_file)
print(translittered)
print(translated)

In [ ]:
kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_02.jpg'
# transcode = KannadaToHindi()
translated, translittered = transcode.TranslateKannadaImageToHindi(kannada_img_file)
print(translittered)
print(translated)

In [ ]:
kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_03.jpg'
# transcode = KannadaToHindi()
translated, translittered = transcode.TranslateKannadaImageToHindi(kannada_img_file)
print(translittered)
print(translated)

In [ ]:
kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_04.jpg'
# transcode = KannadaToHindi()
translated, translittered = transcode.TranslateKannadaImageToHindi(kannada_img_file)
print(translittered)
print(translated)

In [ ]:
# from polyglot.downloader import downloader
# print(downloader.supported_languages_table("transliteration2"))